In [2]:
import os
import os.path as op

import numpy as np

from PIL import Image
from tqdm import tqdm

os.environ['CUDA_VISIBLE_DEVICES']='0'
import easyocr

In [12]:
root_dir = '/hdd2/sergeishutov/imagecaption/CNMT'
feat_path = 'CNMT/data/my_data/features'

npy_path = 'CNMT/data/imdb/imdb_test_filtered_by_image_id.npy'
npy_path = 'CNMT/data/my_data/imdb/imdb_test_my.npy'
npy_filepath = op.join(root_dir, npy_path)
test_imdb = np.load(npy_filepath, allow_pickle=True)
test_imdb[0]

{'date created': '24-NOV-2021'}

In [13]:
test_imdb[-1]

{'image_id': 'pisklia',
 'image_name': 'pisklia.jpg',
 'image_width': 960,
 'image_height': 1280,
 'feature_path': 'pisklia.npy',
 'image_path': 'pisklia.jpg',
 'ocr_tokens': ('PURINA', 'Friskies)', '5', 'Jou', '3ooge', 'PROMISES', 'Qor'),
 'ocr_normalized_boxes': array([[0.190625  , 0.43203124, 0.3125    , 0.45625   ],
        [0.17395833, 0.45703125, 0.35416666, 0.51484376],
        [0.25729167, 0.53984374, 0.278125  , 0.56171876],
        [0.146875  , 0.59296876, 0.16979167, 0.60234374],
        [0.08854166, 0.69375   , 0.18125   , 0.71875   ],
        [0.21993439, 0.5605159 , 0.3206906 , 0.58089036],
        [0.278125  , 0.6171875 , 0.32291666, 0.6171875 ]], dtype=float32),
 'ocr_confidence': (0.9810469320665792,
  0.8598374226868856,
  0.999999880790714,
  0.23523691199411914,
  0.7148659685278661,
  0.9973083918729231,
  0.021062389752532745),
 'obj_normalized_boxes': array([[0.258828  , 0.12873708, 0.8447698 , 0.68176556],
        [0.38090697, 0.        , 0.48324636, 0.19783321]

In [3]:
def normalize_bbox(bbox, im_size):
    #normalize bbox coords from absolute to 0..1
    #a,b  c,b  c,d  a,d  
    #xmin,ymin xmax,ymin xmax,ymax xmin,ymax
    assert len(bbox) == 4 and len(bbox[0]) == 2
    w, h = im_size
    xmin = bbox[0][0] / w
    xmax = bbox[2][0] / w
    ymin = bbox[0][1] / h
    ymax = bbox[2][1] / h
    return (xmin, ymin, xmax, ymax)

In [4]:
reader = easyocr.Reader(['en'])

In [5]:
img_dir = op.join(root_dir, 'images')
im_list = sorted(os.listdir(img_dir))

images_info = [{'date created': '24-NOV-2021'}]

for im_filename in tqdm(im_list):
    im_filepath = op.join(img_dir, im_filename)

    # im = Image.open(im_filepath)
    # w, h = im.width, im.height
    # im.close()

    feat_npy_name = im_filename.split('.')[0] + '_info.npy'
    feat_npy_filepath = op.join(root_dir, feat_path, feat_npy_name)

    feat_item = np.load(feat_npy_filepath, allow_pickle=True).item()
    # assert feat_item['image_width'] == w and feat_item['image_height'] == h
    w, h = feat_item['image_width'], feat_item['image_height']
    
    obj_boxes = feat_item['bbox']  # N x [xmix, ymin, xmax, ymax]
    obj_boxes[:, ::2] /= w
    obj_boxes[:, 1::2] /= h
    obj_normalized_boxes = obj_boxes
    
    ocr_output = reader.readtext(im_filepath)
    if ocr_output:
        ocr_boxes, ocr_tokens, ocr_confidence = list(zip(*ocr_output))
        ocr_normalized_boxes = np.asarray([normalize_bbox(t, (w, h)) for t in ocr_boxes], dtype=np.float32)
    else:
        continue
        #ocr_normalized_boxes = np.zeros((1, 4), dtype=np.float32)
        #ocr_tokens, ocr_confidence = [' '], [1e-5]

    img_id = im_filename.split('.')[0]

    img_info = {
        'image_id': img_id,
        'image_name': img_id,
        'image_width': w,
        'image_height': h,
        'feature_path': img_id + '.npy',
        'image_path': im_filename,
        'ocr_tokens': ocr_tokens,
        'ocr_normalized_boxes': ocr_normalized_boxes,
        'ocr_confidence': ocr_confidence,
        'obj_normalized_boxes': obj_normalized_boxes,
        'caption_id': 300000000,
        'set_name': 'test',
    }
    images_info.append(img_info)


100%|██████████| 7/7 [00:01<00:00,  6.19it/s]


In [6]:
np.save('data/my_data/imdb/imdb_test_my.npy', images_info)
# az = np.load('../tmp.npy', allow_pickle=True)

In [7]:
images_info[5]['ocr_normalized_boxes'] #.reshape(-1,4)

array([[0.10493827, 0.48026314, 0.15185185, 0.57236844],
       [0.27283952, 0.4890351 , 0.4888889 , 0.5964912 ],
       [0.17530864, 0.5701754 , 0.21975309, 0.5877193 ],
       [0.10493827, 0.54605263, 0.14938271, 0.68201756],
       [0.2691358 , 0.5548246 , 0.49506173, 0.75438595],
       [0.37777779, 0.7719298 , 0.48518518, 0.84210527],
       [0.00617284, 0.83114034, 0.09012346, 0.97587717]], dtype=float32)

In [8]:
images_info[4].keys()

dict_keys(['image_id', 'image_name', 'image_width', 'image_height', 'feature_path', 'image_path', 'ocr_tokens', 'ocr_normalized_boxes', 'ocr_confidence', 'obj_normalized_boxes', 'caption_id', 'set_name'])

In [ ]:
os.makedirs('', exist_ok=True)

In [9]:
from sgg_bench.tools.demo.demo_image_oscar import VinVLDetector

ModuleNotFoundError: No module named 'scene_graph_benchmark'